In [9]:
import codecs
import json
import jieba
from os import listdir
from os.path import isfile, isdir, join
import sys

import scene_cal
import time_cal
import info
import rewrite
from hanziconv import HanziConv

sys.setrecursionlimit(1000000)

if __name__ == '__main__':
    print("Please enter the following story format:")
    print('{')
    print("title: XXXX")
    print("special person names: {Y1, Y2, Y3}")
    print("special location names: {Loc1, Loc2}")
    print("special object names: {O1, O2, O3}")
    print("story body:"+"\n"+"line1"+"\n"+"line2"+"\n"+"…."+"\n"+"linen.")
    print('}')
    
    print()
    path = input('File path:') # test_input/三隻小豬_input
    print()
    
    # read file
    jieba.set_dictionary('data/dict.txt.big')
    story = codecs.open(path, 'r', 'utf-8').read()
    story = story.replace('「', '“').replace('」', '”')
    title = story.split('\n')[0].split('title: ')[1].replace('\r', '')
    name = story.split('\n')[1].split('{')[1].split('}')[0].replace('}\r', '').split(', ')
    location = story.split('\n')[2].split('{')[1].split('}')[0].replace('}\r', '').split(', ')
    # object_name = story.split('\n')[3].split('{')[1].replace('}\r', '').split(', ')
    text = story.split('story body:')[1][2:].replace('\r', '')
    
    f_name = open("data/character_dict.txt", "a")
    f_loc = open("data/location_dict.txt", "a")
    for lines in name:
        f_name.writelines('\n'+lines)
    for lines in location:
        f_loc.writelines('\n'+lines)
    f_name.close()
    f_loc.close()
    
    print(title)
    print(name)
    print(location)
    print(text[0:500],'........')
    
    # build up the script
    cut_word = scene_cal.cuttest(text)
    temp_scene, temp_scene_num = scene_cal.split_scene(cut_word, text)
    scene, scene_num = scene_cal.check_whether_merge(temp_scene, temp_scene_num)
    all_characters = []
    scene_sum = []
    scene_idx = 1
    
    for i in range(scene_num):
        text_simple = HanziConv.toSimplified(scene[i])
        new_text, original_words, change_words = rewrite.rewrite(text_simple) # rewrite content
        new_text = HanziConv.toTraditional(new_text)
        
        s = info.Scene(scene[i])
        s1 = info.Scene(new_text)
        scene_character = s.get_character()
        print(scene_character)
        if len(scene_character)!=0:
            for i in range(len(scene_character)):
                for j in range(len(original_words)):
                    if scene_character[i] == original_words[j]:
                        scene_character[i] = scene_character[i].replace(scene_character[i], change_words[j])
        scene_location = s1.get_location()
        scene_content = s1.get_content()
        scene_time = time_cal.extract_time(new_text)

        scene_dict = {
            "scene": scene_idx,
            "location": scene_location,
            "time": scene_time,
            "character": scene_character,
            "contents": scene_content
        }
        all_characters += scene_character
        scene_sum.append(scene_dict)

        scene_idx += 1

    all_characters = list(set(all_characters))
    
    # write output file
    story_dict = {
        "title": title,
        "scenes": scene_num,
        "all_characters": all_characters,
        "scene": scene_sum
    }

    json_str = json.dumps(story_dict, indent=4, ensure_ascii=False)
    with open('test_output/' + title + '_output.json', 'w', encoding='utf-8') as json_file:
        json_file.write(json_str)

Please enter the following story format:
{
title: XXXX
special person names: {Y1, Y2, Y3}
special location names: {Loc1, Loc2}
special object names: {O1, O2, O3}
story body:
line1
line2
….
linen.
}

File path:test_input/text2


Building prefix dict from /mnt/nas/yangyachi/ai_iterature_term_project/term_project3_v2/data/dict.txt.big ...
I0613 14:52:30.999822 140238400550720 __init__.py:111] Building prefix dict from /mnt/nas/yangyachi/ai_iterature_term_project/term_project3_v2/data/dict.txt.big ...
Loading model from cache /tmp/jieba.ud829696f0cc60ca4174b11566884d37d.cache
I0613 14:52:31.002885 140238400550720 __init__.py:131] Loading model from cache /tmp/jieba.ud829696f0cc60ca4174b11566884d37d.cache



補鞋匠的奇特經歷
['馬魯夫', '法蒂瑪', '惡婆']
['古埃及', '開羅城', '清真寺', '補鞋攤點']
從前，古埃及首都開羅城有個補鞋匠，叫馬魯夫，他老婆叫法蒂瑪，可是人們都把她叫做惡婆。為什麼叫她惡婆呢? 因為她為人陰險毒辣，好吃懶做。她在家裡從不幹活兒，不論什麼家務活，她總是支使丈夫馬魯夫去幹。馬魯夫忙裡忙外，整天累得要死要活，還撈不著什麼好兒。惡婆不僅無休止地支使他，而且天天對他挑魚撥刺兒的挑毛病，老是嫌他這沒幹好，那沒辦到，她自己不干，卻厚顏無恥地罵他懶散，不成器，沒出息。就這樣，“惡婆”在丈夫頭上作威作福，不把他當人看待。
馬魯夫是個忠厚老實。安分守己的人，又很愛面子。他認為家醜不可外揚，因而對法蒂瑪的惡行逆施，一直是忍耐容讓。他的家境不好，窮得叮噹響，為了養活老婆，維持這個家庭，他不時地告誡自己要拚命幹活掙錢，買點日常家用品。家裡天天都是等米下鍋， 哪有工夫去和老婆評理?
但是法蒂瑪卻不理這個茬兒，她一點兒也不替馬魯夫著想，她心裡只有她自己。當馬魯夫掙了錢的時候，她就馬上胡吃海塞，買一些奢侈品，把錢花個淨光，根本不考慮如何節省著過日子; 當馬魯夫掙的錢少，或者沒掙到什麼錢的時候，她就故意摔碟子，砸碗， 給馬魯夫臉色看，也不考慮他勞累了一天，讓他幹活，甚至不給他做飯吃。
一天， ........


Loading model cost 0.778 seconds.
I0613 14:52:31.780568 140238400550720 __init__.py:163] Loading model cost 0.778 seconds.
Prefix dict has been built succesfully.
I0613 14:52:31.781222 140238400550720 __init__.py:164] Prefix dict has been built succesfully.


#####scene
從前，古埃及首都開羅城有個補鞋匠，叫馬魯夫，他老婆叫法蒂瑪，可是人們都把她叫做惡婆。為什麼叫她惡婆呢?因為她為人陰險毒辣，好吃懶做。她在家裡從不幹活兒，不論什麼家務活，她總是支使丈夫馬魯夫去幹。馬魯夫忙裡忙外，整天累得要死要活，還撈不著什麼好兒。惡婆不僅無休止地支使他，而且天天對他挑魚撥刺兒的挑毛病，老是嫌他這沒幹好，那沒辦到，她自己不干，卻厚顏無恥地罵他懶散，不成器，沒出息。
#####scene
['從前，古埃及首都開羅城有個補鞋匠，叫馬魯夫，他老婆叫法蒂瑪，可是人們都把她叫做惡婆。為什麼叫她惡婆呢?因為她為人陰險毒辣，好吃懶做。她在家裡從不幹活兒，不論什麼家務活，她總是支使丈夫馬魯夫去幹。馬魯夫忙裡忙外，整天累得要死要活，還撈不著什麼好兒。惡婆不僅無休止地支使他，而且天天對他挑魚撥刺兒的挑毛病，老是嫌他這沒幹好，那沒辦到，她自己不干，卻厚顏無恥地罵他懶散，不成器，沒出息。']
[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O-E', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O-E', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O-E', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O-E', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

...

cnn(
  (embed): embed(
    (char_embed): embed_cnn(
      (embed): Embedding(1510, 50, padding_idx=0)
      (conv): ModuleList(
        (0): Conv2d(1, 50, kernel_size=(3, 50), stride=(1, 1))
      )
      (dropout): Dropout(p=0.5)
      (fc): Linear(in_features=50, out_features=150, bias=True)
    )
    (word_embed): Embedding(751, 150, padding_idx=0)
  )
  (conv): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(2, 300), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(3, 300), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(4, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=300, out_features=154, bias=True)
  (softmax): LogSoftmax()
)
loading DialogueClassification/save_model_ch/model.epoch3000
saved model: epoch = 3000, loss = 0.012693
('我發覺，馬魯夫，清晨跟著給我蘿蔔一大筆藉分花賣些肯納凡(一種獨特酸甜點，舉例來發覺是用細麵條加糖漿或麥芽糖塑膠的。這是開羅人的慶典的食品。)，我很決心愛不肉掉掉完掉完草莓來作的那種。', None, '公主', 0.9343167543411255)
公主 : 我發覺，馬魯夫，清晨跟著給我蘿蔔一大筆藉分花賣些肯納凡(一種獨特酸甜點，舉例來發覺是用細麵條加糖漿或麥芽糖塑膠的。這是開羅人的慶典的食品。)，我很決心愛不肉掉

cnn(
  (embed): embed(
    (char_embed): embed_cnn(
      (embed): Embedding(1510, 50, padding_idx=0)
      (conv): ModuleList(
        (0): Conv2d(1, 50, kernel_size=(3, 50), stride=(1, 1))
      )
      (dropout): Dropout(p=0.5)
      (fc): Linear(in_features=50, out_features=150, bias=True)
    )
    (word_embed): Embedding(751, 150, padding_idx=0)
  )
  (conv): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(2, 300), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(3, 300), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(4, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=300, out_features=154, bias=True)
  (softmax): LogSoftmax()
)
loading DialogueClassification/save_model_ch/model.epoch3000
saved model: epoch = 3000, loss = 0.012693
('多瞭不必蘿蔔一大筆藉分花賣，隻蘿蔔一大筆藉分花賣五塊一大筆藉分花賣的吧。', None, '公主', 0.5157256126403809)
馬魯夫 : 多瞭不必蘿蔔一大筆藉分花賣，隻蘿蔔一大筆藉分花賣五塊一大筆藉分花賣的吧。
可是以前我這兒糖漿用完瞭，不過麥芽糖拉到是有的是，實際上麥芽糖的比糖漿的還要愛不肉掉掉到完掉完呢，你讓她加水著麥芽糖愛不肉掉掉完掉完，也是一樣的。
loading DialogueClassi

saved model: epoch = 3000, loss = 0.012693
('還必跟我頂嘴!', None, '大拇哥', 0.7100857496261597)
馬魯夫 : 還必跟我頂嘴!
你不來著，不是加水糖漿的肯納凡我是一口也不愛不肉掉掉完掉完的!你這個白癡，快給我滾齣去!今兒個你非得給我蘿蔔藉分花賣跟著加水糖漿的肯納凡不宜，否則我就跟你不決心完!
loading DialogueClassification/training_data_ch.char_to_idx
loading DialogueClassification/training_data_ch.word_to_idx
loading DialogueClassification/training_data_ch.tag_to_idx
cnn(
  (embed): embed(
    (char_embed): embed_cnn(
      (embed): Embedding(1510, 50, padding_idx=0)
      (conv): ModuleList(
        (0): Conv2d(1, 50, kernel_size=(3, 50), stride=(1, 1))
      )
      (dropout): Dropout(p=0.5)
      (fc): Linear(in_features=50, out_features=150, bias=True)
    )
    (word_embed): Embedding(751, 150, padding_idx=0)
  )
  (conv): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(2, 300), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(3, 300), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(4, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=300, out_featu

cnn(
  (embed): embed(
    (char_embed): embed_cnn(
      (embed): Embedding(1510, 50, padding_idx=0)
      (conv): ModuleList(
        (0): Conv2d(1, 50, kernel_size=(3, 50), stride=(1, 1))
      )
      (dropout): Dropout(p=0.5)
      (fc): Linear(in_features=50, out_features=150, bias=True)
    )
    (word_embed): Embedding(751, 150, padding_idx=0)
  )
  (conv): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(2, 300), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(3, 300), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(4, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=300, out_features=154, bias=True)
  (softmax): LogSoftmax()
)
loading DialogueClassification/save_model_ch/model.epoch3000
saved model: epoch = 3000, loss = 0.012693
('首席最高原告大人，如果我或許責罵瞭她。拔除瞭她的膝部，那麼，我猶豫接倍均受任何謹慎的受罰!可是真正的狀況發覺明瞭…', None, '公主', 0.8573171496391296)
馬魯夫 : 首席最高原告大人，如果我或許責罵瞭她。拔除瞭她的膝部，那麼，我猶豫接倍均受任何謹慎的受罰!可是真正的狀況發覺明瞭…
如果不是暗自的街坊鄰居們前去勸導，她是不但可能纔必須善罷甘休的。
loading DialogueClass

saved model: epoch = 3000, loss = 0.012693
('這迴你女友把你告到另一個首席最高原告那兒去瞭， 這是我們的首席最高原告傳你，快跟我們停下來吧!', None, '公主', 0.9058332443237305)
公主 : 這迴你女友把你告到另一個首席最高原告那兒去瞭， 這是我們的首席最高原告傳你，快跟我們停下來吧!
我們不是先一由首席最高原告勸解。和好瞭嗎? 你怎麼又到這兒來告我呢?
loading DialogueClassification/training_data_ch.char_to_idx
loading DialogueClassification/training_data_ch.word_to_idx
loading DialogueClassification/training_data_ch.tag_to_idx
cnn(
  (embed): embed(
    (char_embed): embed_cnn(
      (embed): Embedding(1510, 50, padding_idx=0)
      (conv): ModuleList(
        (0): Conv2d(1, 50, kernel_size=(3, 50), stride=(1, 1))
      )
      (dropout): Dropout(p=0.5)
      (fc): Linear(in_features=50, out_features=150, bias=True)
    )
    (word_embed): Embedding(751, 150, padding_idx=0)
  )
  (conv): ModuleList(
    (0): Conv2d(1, 100, kernel_size=(2, 300), stride=(1, 1))
    (1): Conv2d(1, 100, kernel_size=(3, 300), stride=(1, 1))
    (2): Conv2d(1, 100, kernel_size=(4, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5)
  (fc): Lin

In [10]:
import json

with open('test_output/補鞋匠的奇特經歷_output.json') as f:
    d = json.load(f)

#script format
print('Story title: '+d.get('title'))
print()
for i in range(len(d.get('scene'))):
    print()
    print('Scene '+str(d.get('scene')[i].get('scene'))+'. 時 / '+d.get('scene')[i].get('time')+' 景 / '+d.get('scene')[i].get('location'))
    print()
    for j in range(len(d.get('scene')[i].get('contents'))):
        sentence = d.get('scene')[i].get('contents')[j]
        if ':' not in sentence:
            print('△ '+d.get('scene')[i].get('contents')[j])
        else :
            print(d.get('scene')[i].get('contents')[j])
        print()

Story title: 補鞋匠的奇特經歷


Scene 1. 時 / 白天 景 / 家

△ 從前，古羅馬舊都大馬士革城後有個補鞋匠，叫馬魯夫，他老公叫阿拔斯，可是人們都把她是從惡婆。為什麼叫她惡婆呢?因為她為人陰險毒辣，好吃懶做。她在店裏從不乾活兒，不論什麼傢務活，她似乎支使未婚夫馬魯夫去乾。馬魯夫忙裏忙外，整天纍得要死要活，還帶魚肉不著什麼好兒。惡婆不僅無休止地支使他，而且天天對他挑帶魚充作刺兒的挑毛病，老是斥他這不用乾好，那不用辦到，她自己不乾，卻厚顔無恥地破口大罵他散漫，不成器，不用齣息。


Scene 2. 時 / 白天 景 / 家

馬魯夫 : 惡婆

△ 就這樣，在女兒肩上作威作福，不把他當人忽略。

△ 馬魯夫是個忠厚老實。

△ 安分守己的人，又很愛麵子。

△ 他相信傢醜不可外揚，因而對穆薩的罪人逆施，多年來是忍耐容讓。

△ 他的糊口太差，愚得叮當響，為瞭窮睏太太，持續這個普通傢庭，他偶而地勸告自己要拚命乾活養傢，買點日常傢用具。

△ 店裏天天都是等米下鍋，哪有工夫去和太太評理?

△ 但是穆薩卻不理這個茬兒，她一點兒也不替馬魯夫著想，她內心隻有她自己。


Scene 3. 時 / 清晨 景 / 櫥窗

△ 當馬魯夫掙瞭一大筆藉分花賣的時候，她就趕緊鬍愛不肉掉掉完掉完海塞，蘿蔔一大筆藉分花賣一些奢侈品，把一大筆藉分花賣蘿蔔個淨光，實際上不先繼續次此後顧慮如何節約著過日子;

△ 當馬魯夫掙的一大筆藉分花賣少，或者不決心掙到什麼一大筆藉分花賣的時候，她就欺騙不慎碟子，砍傷碗，給馬魯夫樣子看，也不先繼續次此後顧慮他勞纍瞭一天，讓他農夫，甚至不給他來作飯愛不肉掉掉完掉完。

公主 : 我發覺，馬魯夫，清晨跟著給我蘿蔔一大筆藉分花賣些肯納凡(一種獨特酸甜點，舉例來發覺是用細麵條加糖漿或麥芽糖塑膠的。這是開羅人的慶典的食品。)，我很決心愛不肉掉掉完掉完草莓來作的那種。

馬魯夫 : 隻要是你決心愛不肉掉掉完掉完的，我一定但可能纔必須給你蘿蔔一大筆藉分花賣跟著的。盡管我盤子裏一點兒一大筆藉分花賣也不決心有，但是我確實摩西但可能纔必須試圖我給你蘿蔔一大筆藉分花賣迴肯納凡的。

公主 : 我不管摩西一心盡辦法不一心盡辦法你，我隻要求你離去時一定要帶迴肯納凡，對瞭，是糖漿製的最好的肯納凡!要是你兩在手空空地跟著，那我就對你不客精瞭。你等著瞧吧，今清晨我